In [ ]:
from scenarios import (
    generate_hp_cost_projections,
    generate_scenario_attitudes,
    MODES_2020,
    FAST_TRANSITION_MODES_AND_YEARS,
    SLOW_TRANSITION_MODES_AND_YEARS,
)
from config import TECHNOLOGY_COLORS
import pandas as pd
import plotly.express as px

fast_attititudes = generate_scenario_attitudes(
    MODES_2020, FAST_TRANSITION_MODES_AND_YEARS
)
fast_attititudes = fast_attititudes.melt(ignore_index=False)
fast_attititudes["Scenario"] = "Fast"

slow_attitudes = generate_scenario_attitudes(
    MODES_2020, SLOW_TRANSITION_MODES_AND_YEARS
)
slow_attitudes = slow_attitudes.melt(ignore_index=False)
slow_attitudes["Scenario"] = "Slow"

NO_TRANSITION_MODES_AND_YEARS = {
    "Electric furnace": {"end_att": 0.05, "at_year": 2040},
    "Gas furnace": {"end_att": 0.95, "at_year": 2030},
    "Heat pump": {"end_att": 0.05, "at_year": 2030},
    "Oil furnace": {"end_att": 0.95, "at_year": 2030},
    "Wood or wood pellets furnace": {"end_att": 0.109409, "at_year": 2030},
}
MODERATE_MODES_AND_YEARS = {
    "Electric furnace": {"end_att": 0.45, "at_year": 2040},
    "Gas furnace": {"end_att": 0.076923, "at_year": 2030},
    "Heat pump": {"end_att": 0.6, "at_year": 2040},
    "Oil furnace": {"end_att": 0.05, "at_year": 2030},
    "Wood or wood pellets furnace": {"end_att": 0.109409, "at_year": 2030},
}


moderate_attitudes = generate_scenario_attitudes(MODES_2020, MODERATE_MODES_AND_YEARS)
moderate_attitudes = moderate_attitudes.melt(ignore_index=False)
moderate_attitudes["Scenario"] = "Moderate"

no_attitudes = generate_scenario_attitudes(MODES_2020, NO_TRANSITION_MODES_AND_YEARS)
no_attitudes = no_attitudes.melt(ignore_index=False)
no_attitudes["Scenario"] = "Sensitivity"


attititudes = pd.concat([slow_attitudes, fast_attititudes, no_attitudes, moderate_attitudes])

In [ ]:

fig = px.line(attititudes.reset_index(), x="index", y="value", color="variable", facet_row="Scenario", width=700, template="plotly", color_discrete_map=TECHNOLOGY_COLORS)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_yaxis(lambda y: y.update(title="Att. mode"))

In [ ]:


costs = generate_hp_cost_projections(learning_rate=11.1, write_csv=True)
fig = px.line(
    costs.melt(ignore_index=False, var_name="tech").reset_index(),
    x="year",
    y="value",
    color="tech",
    facet_row="variable",
    template="plotly",
    height=800,
    width=600,
)
fig.update_yaxes(
    matches=None,
)


## Raise gas price by increased carbon tax

In [ ]:
import pandas as pd
from data.canada import end_use_prices
old_end_use_prices = end_use_prices.copy()

In [ ]:
from scenarios import update_price_w_new_CT, CT
from functools import partial
new_CT = CT * 2

update_prices = partial(update_price_w_new_CT, new_CT=new_CT)

end_use_prices["Price (ct/kWh)"] = end_use_prices[["Year", "Price (ct/kWh)","Type of fuel","GEO"]].apply(update_prices, axis=1)

In [ ]:
end_use_prices.to_csv("data/canada/residential_GNZ_end-use-prices-2023_ct_per_kWh.csv", index=False)

In [ ]:
import importlib
importlib.import_module("data.canada")

In [ ]:
end_use_prices == old_end_use_prices